<a href="https://colab.research.google.com/github/jpgallego/03MAIR---Algoritmos-de-Optimizacion---2023/blob/main/JesusPedro_GallegoBerciano_Trabajo_Pr%C3%A1ctico_Algoritmos(V2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Jesus Pedro Gallego Berciano <br>
Url: https://github.com/jpgallego/03MAIR---Algoritmos-de-Optimizacion---2023/blob/main/JesusPedro_GallegoBerciano_Trabajo_Pr%C3%A1ctico_Algoritmos(V2).ipynb<br>
Google Colab: https://colab.research.google.com/drive/1dys8lzi_hVRC6OMhFM--8iLN9bgFoygb?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.








                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

**Espacio de soluciones:**

Genero un array de listas. En cada lista se encuentran las tomas a realizar en cada sesión.

**Función objetivo:**

Para cada sesión vemos el número de actores que deben acudir. Por ejemplo, si a la sesión 1 tiene que ir 10 actores para grabas las 6 tomas, el coste de ese día será 10.
El coste total será la suma de los costes de cada sesión. Este coste total es la función objetivo que se pretende minimizar.

**Restricciones**:

La restricción que tiene el problema es que cada sesión puede tener como máximo 6 tomas


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

**Orden de complejidad:**

Para cada iteración la complejidad nos la marca el cálculo del vecindario.

Para ello recorremos cuatro bucles anidados


*   Primer bucle --> Hasta número de sesiones menos 1
*   Segundo bucle --> Desde el valor del contador anterior hasta el número de sesiones.
*   Tercer bucle --> Hasta número de tomas por sesión
*   Cuarto bucle --> Hasta número de tomas por sesión

Por lo que podemos estimar un orden de complejidad O(n^4) siendo n = num tomas total / num tomas máximo por sesión


**Espacio de soluciones**

Para cada solución parcial se genera un vecindario con un total de 360 nuevas soluciones.





#Diseño
- ¿Que técnica utilizo? ¿Por qué?

**Técnica utilizada:**

Método heurístico basado en búsqueda tabú.

La idea era buscar un método heurístico ya que creo que es más conveniente para este tipo de problemas.

Y en concreto me he decantado por la búsqueda tabú porque creo que dada la forma en la que se genera la solución era un método adecuado.

DESARROLLO DEL MODELO

In [13]:
import pandas as pd
import numpy as np
import copy
from time import time

In [14]:
datos=pd.read_csv("datos.csv", sep=";")


In [15]:
def extraer_datos(datos, num_tomas, num_actores):

  # Función que extrae los datos del DataFrame y genera dos arrays
  # tomas_por_actor --> cada lista contiene las tomas en las que participa cada actor
  # actores_por_toma --> cada lista contiene los actores que participan en cada toma

  tomas_por_actor=[]
  actores_por_toma=[]
  for i in range(num_actores):
    # Recorre el DataFrame por columnas
    lista=datos.index[datos.iloc[:,i] == 1].tolist()
    tomas_por_actor.append(lista)
  for i in range(num_tomas):
    lista = []
    for j in range(num_actores):
      # Recorre el DataFrame por filas
      if datos.iloc[i,j] == 1:
        lista.append(j)
    actores_por_toma.append(lista)
  return tomas_por_actor, actores_por_toma

In [16]:
def solucion_inicial(max_tomas, num_tomas):

  # Genera una solución inicial colocando las tomas de forma consecutiva de la 0 a 29

  lista=[]
  solucion = []
  for i in range(num_tomas):
    lista.append(i)
    if len(lista) == max_tomas:
      solucion.append(lista)
      lista = []
    else:
        if i == num_tomas-1:
          solucion.append(lista)

  return solucion

In [17]:
def evalua_solucion(solucion, tomas_por_actor, actores_por_toma):

  # Evalua la solución viendo cuántos actores distintos hay en cada sesión y sumando todos los costes por sesión

  sesion_aux=[]
  coste = 0
  for i in range(len(solucion)):
    sesion_aux=[]
    for j in range(len(solucion[i])):
      if solucion[i][j] > -1:
        sesion_aux+=actores_por_toma[solucion[i][j]]
    sesion=set(sesion_aux)
    coste+=len(sesion)

  return(coste)

In [18]:
def genera_vecindario(solucion):

  # Para generar el vecindario de una solución parcial vamos intercambiando las tomas de dos en dos entre dos días distintos

  vecindario=[]
  movimientos=[]
  for i in range(len(solucion) - 1):
    for j in range(i+1, len(solucion)):
      for k in range(len(solucion[i])):
        for l in range(len(solucion[j])):
          vecino=copy.deepcopy(solucion)
          # Guardamos el nuevo vecino
          vecino[i][k], vecino[j][l] = vecino[j][l], vecino[i][k]
          vecindario.append(vecino)
          # Guardamos el movimiento realizado
          mov = [vecino[i][k], vecino[j][l]]
          movimientos.append(mov)
  return vecindario, movimientos


In [19]:
def busqueda_tabu(datos, iteraciones, tamano_lista_tabu):

    # Inicia el temporizador
    start_time = time()

    # Extrae el número de filas (num_tomas) y columnas (num_actores) del DataFrame
    num_tomas, num_actores = datos.shape
    # Extrae los datos del DataFrame
    tomas_por_actor, actores_por_toma= extraer_datos(datos, num_tomas, num_actores)
    # Genera la solución inicial
    solucion_ini=solucion_inicial(max_tomas_sesion, num_tomas)

    # Inicializa la mejor solución con la solución inicial
    mejor_solucion = copy.deepcopy(solucion_ini)
    mejor_vecino = copy.deepcopy(solucion_ini)
    mejor_coste = evalua_solucion(mejor_solucion, tomas_por_actor, actores_por_toma)
    lista_tabu = []

    for i in range(iteraciones):
       # En cada iteración genera los vecinos de la solución inicial y los movimientos realizados para generar ese vecindario
        vecindario, movimientos = genera_vecindario(mejor_vecino)
        mejor_vecino = None
        mejor_vecino_coste = float('inf')

        #Evalua cada una de las soluciones del vecindario
        for vecino in vecindario:
            coste_vecino = evalua_solucion(vecino, tomas_por_actor, actores_por_toma)
            indice = vecindario.index(vecino)
            if (coste_vecino < mejor_vecino_coste) and (movimientos[indice] not in lista_tabu) and (movimientos[indice][::-1] not in lista_tabu):
              # Si es el mejor vecino y el movimiento no esta en la lista tabú, almacenamos ese vecino como el mejor
              mejor_vecino = vecino
              mejor_vecino_coste = coste_vecino
              mejor_indice = indice

        if mejor_vecino is not None:
          if mejor_vecino_coste < mejor_coste:
            # Almacenamos el mejor de los vecinos encontrado si es la mejor solución encontrada hasta ese momento
            mejor_solucion = mejor_vecino
            mejor_coste = mejor_vecino_coste
            iter=i
            print('[INFO] ---- Mejor solución encontrada:\n', mejor_solucion)
            print('Iteración: ', i, 'Coste: ', mejor_coste, '\n')

          # Almacenamos el movimiento realizado en la lista tabú
          lista_tabu.append(movimientos[mejor_indice])

          # Si se sobrepasa el tamaño de la lista eliminamos el primer movimiento de la lista
          if len(lista_tabu) > tamano_lista_tabu:
            lista_tabu.pop(0)
        if i % 100 == 0 and i > 0:
          print('[INFO] ---- Paso por la iteracion: ', i, '\n')
    elapsed_time = time() - start_time

    presentar_solucion(mejor_solucion, mejor_coste, iter, num_tomas, elapsed_time)

    return

In [20]:
def presentar_solucion(solucion_final, coste, iteracion, num_tomas, elapsed_time):

  # Función que presenta los datos de la mejor solución por pantalla
  solucion=copy.deepcopy(solucion_final)
  for i in range(len(solucion)):
    for j in range(max_tomas_sesion):
      if (i)*max_tomas_sesion + j >= num_tomas:
        break
      solucion[i][j]+=1
  print('\n------------------- SOLUCION ------------------- ')
  for sesion in range (len(solucion)):
    print('Tomas en la sesión', sesion + 1, ':', solucion[sesion])
  print('------------------------------------------------ ')
  print('Esta distribución de tomas tiene un coste de:',coste)
  print('Encontrada en la iteración:', iteracion)
  print("Tiempo de ejecución: %.4f segundos." % elapsed_time)
  print('------------------------------------------------ ')

In [22]:
max_tomas_sesion = 6
num_iteraciones = 500
tamaño_lista = 30

busqueda_tabu(datos, num_iteraciones, tamaño_lista)

[INFO] ---- Mejor solución encontrada:
 [[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11], [24, 13, 14, 15, 16, 17], [18, 19, 20, 21, 22, 23], [12, 25, 26, 27, 28, 29]]
Iteración:  0 Coste:  35 

[INFO] ---- Mejor solución encontrada:
 [[10, 1, 2, 3, 4, 5], [6, 7, 8, 9, 0, 11], [24, 13, 14, 15, 16, 17], [18, 19, 20, 21, 22, 23], [12, 25, 26, 27, 28, 29]]
Iteración:  1 Coste:  34 

[INFO] ---- Mejor solución encontrada:
 [[10, 21, 2, 3, 4, 5], [6, 7, 8, 9, 0, 11], [24, 13, 14, 15, 16, 17], [18, 19, 20, 1, 22, 23], [12, 25, 26, 27, 28, 29]]
Iteración:  2 Coste:  33 

[INFO] ---- Mejor solución encontrada:
 [[10, 14, 2, 3, 4, 5], [6, 7, 8, 9, 0, 11], [24, 13, 21, 15, 16, 17], [18, 19, 20, 1, 22, 23], [12, 25, 26, 27, 28, 29]]
Iteración:  3 Coste:  32 

[INFO] ---- Mejor solución encontrada:
 [[10, 14, 2, 3, 4, 5], [25, 7, 8, 9, 0, 11], [24, 13, 21, 15, 16, 17], [18, 19, 20, 1, 22, 23], [12, 6, 26, 27, 28, 29]]
Iteración:  4 Coste:  31 

[INFO] ---- Mejor solución encontrada:
 [[10, 14, 2, 3, 4, 5